# **Import used libraries**


In [ ]:
from keras.models import Model
import tensorflow as tf
from keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, GlobalAveragePooling2D, Dense, Dropout
from keras.layers import concatenate
from keras.applications.inception_v3 import InceptionV3
import urllib.request
import requests
from PIL import Image
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from google.colab import drive
from keras.applications.inception_v3 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator

# **Connecting to Google Drive**


Use `flush_and_unmount` and `force_remount=True` for safe connection.

---

In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

# **Training and validation process**




In [ ]:
# Constants
num_classes = 8
input_shape = (1024, 1024, 3)
output_fc_units = 2048

In [ ]:
# Load pre-trained InceptionV3 model without top classification layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

In [ ]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = True

In [ ]:
# Add custom classification layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)

# New fully-connected layer of size N x 2048
output_fc_layer = Dense(output_fc_units, activation='relu')(x)
# Softmax and classification output layer
softmax_layer = Dense(num_classes, activation='softmax')(output_fc_layer)

In [ ]:
# Create the new model
model = Model(inputs=base_model.input, outputs=softmax_layer)

In [ ]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.015)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.load_weights("/content/drive/MyDrive/BSc Project/Codes/TF_googlent_V2.h5")

In [ ]:
# Data preparation
train_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/BSc Project/Skin Cancer Datasets/ISIC_2019_Dataset/Trainning Dataset',
    target_size=(1024, 1024),
    batch_size=16,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/BSc Project/Skin Cancer Datasets/ISIC_2019_Dataset/Validation Dataset',
    target_size=(1024, 1024),
    batch_size=16,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
# model.fit(train_generator, epochs=10, validation_data=test_generator)

# **Getting inference from model**

In [ ]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(input_shape[0], input_shape[1]))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

In [ ]:
# Make predictions on a new image
def predict_image(image_path):
    processed_image = preprocess_image(image_path)
    predictions = model.predict(processed_image)
    class_labels = ['actinic keratosis','basal cell carcinoma','dermatofibroma','melanoma', 'nevus', 'pigmented benign keratosis', 'squamous cell carcinoma','vascular lesion']

    decoded_predictions = [
        (class_labels[i], prob) for i, prob in enumerate(predictions[0])
    ]

    decoded_predictions.sort(key=lambda x: x[1], reverse=True)

    for pred_class, pred_prob in decoded_predictions:
        print(f'{pred_class}: {pred_prob*100:.2f}%')

> **Load model**

In [ ]:
import os
from keras.models import load_model
os.chdir("/content/drive/MyDrive/BSc Project/Codes")
model.save_weights('TF_googlent_V2.h5')

In [ ]:
image_path = "/content/drive/MyDrive/BSc Project/Skin Cancer Datasets/ISIC_2019_Dataset/Trainning Dataset/basal cell carcinoma/ISIC_0061056.jpg"
# predict_image(image_path)

# **Evaluation Process**

In [ ]:
eval_results = model.evaluate(test_generator)

In [ ]:
print("Loss:", eval_results[0])
print("Accuracy:", eval_results[1])